In [1]:
import os
import openai
import numpy as np 
import pandas as pd
import time 
import pdfkit
import re
import copy
from tqdm import tqdm
from itertools import product
openai.organization = os.environ.get("OPENAI_ORGANIZATION")
openai.api_key = os.environ.get("OPENAI_API_KEY")
pd.set_option('display.max_rows', 500)
openai.Model.retrieve('gpt-3.5-turbo')

C:\Users\lucid\AppData\Roaming\Python\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lucid\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\lucid\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


<Model model id=gpt-3.5-turbo at 0x1d97f9e74a0> JSON: {
  "created": 1677610602,
  "id": "gpt-3.5-turbo",
  "object": "model",
  "owned_by": "openai",
  "parent": null,
  "permission": [
    {
      "allow_create_engine": false,
      "allow_fine_tuning": false,
      "allow_logprobs": true,
      "allow_sampling": true,
      "allow_search_indices": false,
      "allow_view": true,
      "created": 1683852530,
      "group": null,
      "id": "modelperm-C34BeeteYvuLNoa893aJRcnp",
      "is_blocking": false,
      "object": "model_permission",
      "organization": "*"
    }
  ],
  "root": "gpt-3.5-turbo"
}

In [2]:
dfp = pd.read_csv('ExplodedPositivesEV.csv').dropna()
dfn = pd.read_csv('ExplodedNegativesEV.csv').dropna()
dfw = pd.read_csv('ExplodedWishlistEV.csv').dropna()

In [3]:
def semantic_condenser2(df, context):
    master_list = []
    index_exceptions = []
    #Taking 500 first pass features at a time to stay with 4096 context length

    features = df["Features"].tolist()
    ft = len(features)
    #Accounting for number of features under 500, between the exact multiples of 500 etc. 
    qt = len(features) // context
    rem = len(features) % context
    if rem == 0:
        iterations = qt
    else:
        iterations = qt + 1
    ind = 0
    if qt == 0:
        ind2 = len(features)
    else:
        ind2 = context
        #print('Model: ', i)
        #print('Model Occurences: ', ft)
        #print('Iterations', iterations)
    for j in tqdm(range(iterations)):
        try:
            content = openai.ChatCompletion.create(
            model = 'gpt-3.5-turbo',
            messages=[
        {"role": "system", "content": "You are a helpful assistant that performs the specific task being asked."},
        {"role": "user", "content": "Given a list of words in the following list which contain many words with similar semantic "\
         "meaning, condense those words into a smaller list of words which represent the same meaning. The words are related to "\
         "vehicle features. Make each word or feature into a bullet point. Focus purely on vehicle features like Fuel Efficiency, Spacious Interiors etc. "\
         "Try to make each feature as distinct as possible from the others. If similar, try to group them together into a "\
         "single feature. Avoid long bullet points. Do not number the bullets. Do not add anything else to the response like "\
         "a Note or a warning. List:  " + (', ').join(features[ind:ind2])}
         ],
        temperature = 1,
        max_tokens= 1000,
        frequency_penalty = 0,
        presence_penalty = 0.9,
        #n=3
            ).get("choices")[0]['message']['content'].split('\n')
        
            ret = [x.replace('- ', '') for x in content]
        
        #Chaining and further compression at this stage does not work well. It seems to ignore making each word/feature into a 
        #separate bullet and instead uses and to group them together which isn't what we want.
        #content2 = openai.ChatCompletion.create(
        #    model = 'gpt-3.5-turbo',
        #    messages=[
        #{"role": "system", "content": "You are a helpful assistant that performs the specific task being asked."},
        #{"role": "user", "content": "Given a list of words in the following list which contain many words with similar semantic "\
        # "meaning, condense those words into a smaller list of words which represent the same meaning. The words are related to "\
        # "vehicle features. Do not add anything else to the response. Make each word or feature into a bullet point. "\
        # "Try to make each feature as distinct as possible from the others. If similar, try to group them together into a "\
        # "single feature. List:  " + (', ').join(ret)}
        # ],
        #temperature = 1,
        #max_tokens= 1000,
        #frequency_penalty = 0,
        #presence_penalty = 0.9,
        #n=3
        #    ).get("choices")[0]['message']['content'].split('\n')
        
        #ret2 = [x.replace('- ', '') for x in content2]
                #print('Index1: ', ind)
                #print('Index2: ', ind2)
            ind = ind + context
            if (ind2 + context) > ft:
                ind2 = ft
            else:
                ind2 =  ind2 + context
            master_list.append(ret)
            
        except Exception as e:
            print(e)
            index_exceptions.append((ind, ind2))
            continue

        
    #Conversion into a data format that's identical to the orginal input. i.e, a dataframe with relevant column names and no 
    #nested data structures. 
    master_list2 = []
    for i in master_list:
        master_list2.append(', '.join(i))
    level1 = pd.DataFrame(master_list2, columns = ["Features"])
    level1e = pd.DataFrame(level1.assign(Features = level1['Features'].str.split(',')).explode('Features'))
    #level1e = pd.DataFrame(level1e.assign(Features = level1e['Features'].str.split(' and ')).explode('Features')) 
    level1e = pd.DataFrame(level1e.assign(Features = level1e['Features'].str.split('/')).explode('Features')) 
    level1e['Word Count'] = level1e['Features'].str.split(' ').str.len()
    level1e = level1e.drop_duplicates()
    return (level1e, index_exceptions)

In [4]:
level1_masterp = semantic_condenser2(dfp, 300)

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [24:25<00:00, 19.80s/it]


In [6]:
level1_masterpos = level1_masterp[0]

In [7]:
level1_mastern = semantic_condenser2(dfn, 250)
level1_masterneg = level1_mastern[0]

 34%|███████████████████████████▍                                                     | 42/124 [13:59<22:01, 16.12s/it]

That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID fdaa987b27c5b75564efeb9a3c14ee65 in your message.)


 60%|████████████████████████████████████████████████▉                                | 75/124 [25:39<20:54, 25.61s/it]

That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 8df6a7a9565b4c52dd6dc6a09b428528 in your message.)


100%|████████████████████████████████████████████████████████████████████████████████| 124/124 [44:42<00:00, 21.64s/it]


In [20]:
level1_mastern

(                                              Features  Word Count
 0                       Shorter Range than Competitors           4
 0                                          Feels Heavy           3
 0                                               Pricey           2
 0              Poor gas mileage for a car of this type          10
 0                           Poor response from Genesis           5
 ..                                                 ...         ...
 121   Limited availability of Class 3 charging stat...          11
 121                              Terrible resale value           4
 121                          Bad fuel efficiency (mpg)           5
 121   Limited selection of vehicles with desired fe...           9
 121                         Extremely sensitive system           4
 
 [4944 rows x 2 columns],
 [(10250, 10500), (18250, 18500)])

In [8]:
level1_masterw = semantic_condenser2(dfw, 250)
level1_masterwish = level1_masterw[0]

 22%|█████████████████▊                                                                | 10/46 [05:12<17:59, 29.99s/it]

That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a4d43c41ca0e4ec2024c4dc6d9882612 in your message.)


 30%|████████████████████████▉                                                         | 14/46 [07:11<15:36, 29.25s/it]

That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 5b97d4b9e7b2f790e2809757a296886d in your message.)


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [26:48<00:00, 34.96s/it]


In [9]:
level2_masterp = semantic_condenser2(level1_masterpos, 300)
level2_masterpos = level2_masterp[0]

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [05:27<00:00, 27.28s/it]


In [15]:
level2_mastern = semantic_condenser2(level1_masterneg, 300)
level2_masterneg = level2_mastern[0]

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [07:36<00:00, 26.87s/it]


In [16]:
level2_masterw = semantic_condenser2(level1_masterwish, 300)
level2_masterwish = level2_masterw[0]

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [05:49<00:00, 31.81s/it]


In [17]:
level3_masterp = semantic_condenser2(level2_masterpos, 300)
level3_mastern = semantic_condenser2(level2_masterneg, 300)
level3_masterw = semantic_condenser2(level2_masterwish, 300)
level3_masterpos = level3_masterp[0]
level3_masterneg = level3_mastern[0]
level3_masterwish = level3_masterw[0]

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:15<00:00, 45.18s/it]


In [45]:
#Level4 can be used in this case as the final condensed list as it completely avoids overlap
level4_masterp = semantic_condenser2(level3_masterpos, 300)
level4_mastern = semantic_condenser2(level3_masterneg, 300)
level4_masterw = semantic_condenser2(level3_masterwish, 300)
level4_masterpos = level4_masterp[0]
level4_masterneg = level4_mastern[0]
level4_masterwish = level4_masterw[0]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:13<00:00, 73.89s/it]


In [ ]:
level4_masterpos.to_csv('CondensedPositivesEV_L4.csv')
level4_masterneg.to_csv('CondensedNegativesEV_L4.csv')
level4_masterwish.to_csv('CondensedWishlistEV_L4.csv')